## Contractor Assignment Problem - General LP Code

In [1]:
# import Glop package
from ortools.linear_solver import pywraplp as glp
import lptools as lpt

In [2]:
#Create LP model object
mymodel = glp.Solver('Contractor Assignment',glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [3]:
inf = mymodel.infinity()

                    # (lower bound, upper bound, objective coefficient)
variables = {'WA': (0, inf, 50),    # assign Westside to project A
             'WB': (0, inf, 36),    # assign Westside to project B
             'WC': (0, inf, 16),    # assign Westside to project C
             'FA': (0, inf, 28),    # Federated
             'FB': (0, inf, 30),
             'FC': (0, inf, 18),
             'GA': (0, inf, 35),    # Goliath
             'GB': (0, inf, 32),
             'GC': (0, inf, 20),
             'UA': (0, inf, 25),    # Universal
             'UB': (0, inf, 25),
             'UC': (0, inf, 14)}

                  # (lower bound, upper bound, coefficient list)
constraints = {'A': (1 ,1 , [1,0,0,1,0,0,1,0,0,1,0,0]),    # exactly one contractor assigned to project A
               'B': (1 ,1  , [0,1,0,0,1,0,0,1,0,0,1,0]),    # exactly one contractor assigned to project B
               'C': (1 ,1  , [0,0,1,0,0,1,0,0,1,0,0,1]),    # exactly one contractor assigned to project C
               'Westside':  (-inf ,1 , [1,1,1,0,0,0,0,0,0,0,0,0 ]),    # Westside assigned to at most one project
               'Federated': (-inf ,1 , [0,0,0,1,1,1,0,0,0,0,0,0 ]),    # Federated assigned to at most one project
               'Goliath':   (-inf ,1 , [0,0,0,0,0,0,1,1,1,0,0,0 ]),    # Goliath assigned to at most one project
               'Universal': (-inf ,1 , [0,0,0,0,0,0,0,0,0,1,1,1 ])}    # Universal assigned to at most one project

mymodel.Objective().SetMinimization()

In [4]:
for v in variables:
    (lb,ub,c) = variables[v]
    var = mymodel.NumVar(lb,ub,v)
    mymodel.Objective().SetCoefficient(var,c)

In [5]:
for c in constraints:
    (lb,ub,coeff_lst) = constraints[c]
    constr = mymodel.Constraint(lb,ub,c)
    for (v,coeff) in zip(mymodel.variables(),coeff_lst):
        constr.SetCoefficient(v,coeff)

In [6]:
lpt.print_model(mymodel)

Variables:
WA, WB, WC, FA, FB, FC, GA, GB, GC, UA, UB, UC 

minimize: 50.0*WA + 36.0*WB + 16.0*WC + 28.0*FA + 30.0*FB + 18.0*FC + 35.0*GA + 32.0*GB + 20.0*GC + 25.0*UA + 25.0*UB + 14.0*UC 

Subject To:
A: 1.0*WA + 1.0*FA + 1.0*GA + 1.0*UA = 1.0
B: 1.0*WB + 1.0*FB + 1.0*GB + 1.0*UB = 1.0
C: 1.0*WC + 1.0*FC + 1.0*GC + 1.0*UC = 1.0
Westside: 1.0*WA + 1.0*WB + 1.0*WC <= 1.0
Federated: 1.0*FA + 1.0*FB + 1.0*FC <= 1.0
Goliath: 1.0*GA + 1.0*GB + 1.0*GC <= 1.0
Universal: 1.0*UA + 1.0*UB + 1.0*UC <= 1.0

Bounds:
WA >= 0.0
WB >= 0.0
WC >= 0.0
FA >= 0.0
FB >= 0.0
FC >= 0.0
GA >= 0.0
GB >= 0.0
GC >= 0.0
UA >= 0.0
UB >= 0.0
UC >= 0.0


In [7]:
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f' % mymodel.Objective().Value())
for v in mymodel.variables():
    print('%s = %.2f' % (v.name(),v.solution_value()))

Solution Status = 0
Optimal Value = 69.00
WA = 0.00
WB = 0.00
WC = 1.00
FA = 1.00
FB = 0.00
FC = 0.00
GA = 0.00
GB = 0.00
GC = 0.00
UA = 0.00
UB = 1.00
UC = 0.00


In [8]:
# display all variable information
print('Variable    LB   Value    UB   Reduced Cost')
for v in mymodel.variables():
    print('%8s  %5.1f  %5.1f  %5.1f  %5.2f' % (v.name(),v.lb(),v.solution_value(),v.ub(),v.reduced_cost()))

Variable    LB   Value    UB   Reduced Cost
      WA    0.0    0.0    inf  23.00
      WB    0.0    0.0    inf   9.00
      WC    0.0    1.0    inf   0.00
      FA    0.0    1.0    inf   0.00
      FB    0.0    0.0    inf   2.00
      FC    0.0    0.0    inf   1.00
      GA    0.0    0.0    inf   7.00
      GB    0.0    0.0    inf   4.00
      GC    0.0    0.0    inf   3.00
      UA    0.0    0.0    inf   0.00
      UB    0.0    1.0    inf   0.00
      UC    0.0    0.0    inf  -0.00


In [12]:
# only print nonzero assignment variables
print('Variable    LB   Value    UB   Reduced Cost')
for v in mymodel.variables():
    if v.solution_value() != 0:
        print('%8s  %5.1f  %5.1f  %5.1f  %5.2f' % (v.name(),v.lb(),v.solution_value(),v.ub(),v.reduced_cost()))

Variable    LB   Value    UB   Reduced Cost
      WC    0.0    1.0    inf   0.00
      FA    0.0    1.0    inf   0.00
      UB    0.0    1.0    inf   0.00


In [10]:
#display constraint information
print('Constraint    LB    Value  UB     Dual')
for (c,lhs) in zip(mymodel.constraints(),mymodel.ComputeConstraintActivities()):
    print('%10s  %5.1f  %5.1f  %5.1f  %5.2f' % (c.name(),c.lb(),lhs,c.ub(),c.dual_value()))

Constraint    LB    Value  UB     Dual
         A    1.0    1.0    1.0  28.00
         B    1.0    1.0    1.0  28.00
         C    1.0    1.0    1.0  17.00
  Westside   -inf    1.0    1.0  -1.00
 Federated   -inf    1.0    1.0   0.00
   Goliath   -inf    0.0    1.0   0.00
 Universal   -inf    1.0    1.0  -3.00
